## Knapsack problem

### Create knapsack problem

In [6]:
from QHyper.problems.knapsack import KnapsackProblem

knapsack = KnapsackProblem(max_weight=1, items=[(1, 2), (1, 1)])
print(knapsack)

In [2]:
print(f"Variables: {knapsack.variables}")
print(f"Objective function: {knapsack.objective_function}")
print("Constraints:")
for constraint in knapsack.constraints:
    print(f"    {constraint}")

Variables: (x0, x1, x2)
Objective function: -2*x0 - x1
Constraints:
    1 - x2
    -x0 - x1 + x2


### Use VQA to solve knapsack problem

In [3]:
import numpy as np

from QHyper.solvers.vqa.base import VQA
from QHyper.solvers.vqa import QAOA, HQAOA

In [4]:
tester_config = {
    'pqc': {
        'type': 'wfqaoa',
        'layers': 5,
    }
}

tester = VQA(knapsack, config=tester_config)

In [5]:
solver_config = {
    'pqc': {
        'type': 'hqaoa',
        'layers': 5,
    },
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200,
        'bounds': [(1, 10)]*3+[(0, 2*np.pi)]*10
    },
}
params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2.5, 2.5],
}

vqa = VQA(knapsack, config=solver_config)
best_params = vqa.solve(params_cofing)
print(best_params)
print(tester.evaluate(best_params))

{'angles': array([3.3026823 , 0.09691683, 1.09066138, 1.71285687, 4.16481097,
       1.36254617, 0.43401708, 0.8264723 , 0.03092491, 2.31893417]), 'hyper_args': array([1.60097218, 2.95596484, 5.05887729])}
-0.9371467822716899


In [5]:
solver_config = {
    'pqc': {
        'type': 'qaoa',
        'layers': 5,
    },
    # 'optimizer': {
    #     'type': 'scipy',
    #     'maxfun': 200,
    #     'bounds': [(0, 2*np.pi)]*10
    # },
}

params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2, 2],
}

vqa = VQA(knapsack, config=solver_config)
best_params = vqa.evaluate(params_cofing)

In [7]:
#WFQAOA
print(best_params)
print(tester.evaluate(best_params))

{'angles': array([[6.20194141, 5.82250628, 1.92747774, 4.20412212, 0.        ],
       [5.83373888, 6.28318531, 5.90484631, 6.248782  , 1.66597206]]), 'hyper_args': [1, 2.5, 2.5]}
-1.0037254325767708


In [8]:
#QAOA
print(best_params)
print(tester.evaluate(best_params))

{'angles': array([[6.20194141, 5.82250628, 1.92747774, 4.20412212, 0.        ],
       [5.83373888, 6.28318531, 5.90484631, 6.248782  , 1.66597206]]), 'hyper_args': [1, 2.5, 2.5]}
-1.0037254325767708


In [6]:
#HQAOA
print(best_params)
print(tester.evaluate(best_params))

{'angles': array([2.83711847e+00, 1.97940203e-01, 1.07858283e+00, 1.63047414e+00,
       4.04555732e+00, 1.35028794e+00, 5.49151498e-01, 8.21588171e-01,
       9.77277636e-04, 2.20975660e+00]), 'hyper_args': array([1.31189481, 2.95215395, 3.76936221])}
-0.9657474465012011


{'angles': array([2.83711847e+00, 1.97940203e-01, 1.07858283e+00, 1.63047414e+00,
       4.04555732e+00, 1.35028794e+00, 5.49151498e-01, 8.21588171e-01,
       9.77277636e-04, 2.20975660e+00]), 'hyper_args': array([1.31189481, 2.95215395, 3.76936221])}


In [5]:
print(best_params)

{'angles': array([[1.64230366, 0.08750879, 0.        , 0.12854769, 2.49365561],
       [0.93907822, 0.62299303, 1.10213747, 0.41506598, 1.45096723]]), 'hyper_args': [1, 2.5, 2.5]}


In [5]:
print(best_params)

{'angles': array([[6.20194141, 5.82250628, 1.92747774, 4.20412212, 0.        ],
       [5.83373888, 6.28318531, 5.90484631, 6.248782  , 1.66597206]]), 'hyper_args': [1, 2.5, 2.5]}


In [17]:
print(f"Best results: {best_result}")
print(f"Params used for optimizer: {optimizer_params},\nand params used for hyper optimizer: {hyper_params}")
# We haven't use hyper optimizer and qaoa only optimize angles, hence hyperparameters are the same as provided

Best results: -1.2280236880975992
Params used for optimizer: [[6.20194141 5.82250628 1.92747774 4.20412212 0.        ]
 [5.83373888 6.28318531 5.90484631 6.248782   1.66597206]],
and params used for hyper optimizer: [1.  2.5 2.5]


In [15]:
params_cofing = {
    'angles': optimizer_params,
    'hyper_args': hyper_params,
}
# You can provide VQA with objects instead of cofinguration dict
# Here we create VQA instance, without optimizer, which will results geting just a quantum circuit (no optimization inside)
vqa = VQA(knapsack, QAOA(layers=5), None, WFEval())
best_result, _, _ = vqa.solve(params_cofing)
print(f"Results are the same as above: {best_result}")

Results are the same as above: -1.0037254325767708


#### Using hyper optimizers

In [7]:
from QHyper.optimizers.random import Random

solver_config = {
    'pqc': {
        'type': 'qaoa',
        'layers': 5,
    },
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200
    },
    'eval_func': {
        'type': 'expval'
    }
}
params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2.5, 2.5],
}
vqa = VQA(knapsack, config=solver_config)

random = Random(processes=5, number_of_samples=100, bounds=[[1, 10], [1, 10], [1, 10]])
best_result_h, optimizer_params_h, hyper_params_h = vqa.solve(params_cofing, random)

100%|██████████| 100/100 [26:24<00:00, 15.84s/it]


In [8]:
print(f"Best results from hyper optimizer: {best_result_h}")
print(f"Params used for optimizer: {optimizer_params_h},\nand params used for hyper optimizer: {hyper_params_h}")

Best results from hyper optimizer: -41.0438258707136
Params used for optimizer: [[0.0238681  5.89321814 6.17761962 0.         2.6826312 ]
 [0.         0.55771513 0.13841942 6.11464352 6.0394204 ]],
and params used for hyper optimizer: [9.41620504 4.94209924 2.11245363]


In [9]:
params_cofing = {
    'angles': optimizer_params_h,
    'hyper_args': hyper_params_h,
}
# You can provide VQA with objects instead of cofinguration dict
# Here we create VQA instance, without optimizer, which will results geting just a quantum circuit (no optimization inside)
vqa = VQA(knapsack, QAOA(layers=5), None, WFEval())
best_result, _, _ = vqa.solve(params_cofing)
print(f"Results are the same as above: {best_result}")

Results are the same as above: -0.06004883742587043
